In [1]:
import time
from datetime import datetime, timedelta
import openai
import json
import pandas as pd
from datetime import datetime

class CacheStack:
    
    def __init__(self, max_size, item_length):
        self.stack = []
        self.max_size = max_size
        self.item_length = item_length
    
    def push(self, item):
        if len(self.stack) >= self.max_size:
            self.stack.pop()
        if len(item) > self.item_length:
            item = item[:self.item_length]
        self.stack.insert(0, item)

    
    def pop(self):
        if len(self.stack) == 0:
            raise Exception("Stack is empty")
        return self.stack.pop(self.max_size - 1)
    
    def is_empty(self):
        return len(self.stack) == 0
    
    def size(self):
        return len(self.stack)
    
    def peek(self):
        if len(self.stack) == 0:
            raise Exception("Stack is empty")
        return self.stack[0]

    def convert_list_to_json(self):
        res = {"context": [item for item in self.stack]}
        return json.dumps(res)




# Function query GPT-4
def get_responde_from_gpt4(sys, usr, assist, tmp=0):
    response = openai.ChatCompletion.create(
    model="gpt-4", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
    #model="gpt-3.5-turbo",
    temperature = tmp,
    messages=[
        {"role": "system", "content": sys},
        {"role": "user", "content": usr},
        {"role": "assistant", "content": assist}
    ])
    res = (response['choices'][0]['message']['content'])
    return res


# Function to determine if the query is out of the scope of this app
def is_a_valid_qry(qry):
    sys = "You are a helpful assistant that will stop queries not related to the scope of this app."
    usr = """
    This app can answer questions related to historic data containing dates, 
    products, quantities, and prices. All questions that can be answered using
    this type of data like querying for products, suggesting products based on 
    former buys, etc. are valid.
    Answer yes if the following question is within the scope of the application, otherwise answer no.
    Place the yes or no at the beginning of your response please.

    """
    assist = qry
    res = get_responde_from_gpt4(sys, usr, assist)
    return res
        
""" Function to determine which days the query might be referred to. This is
required to be able to extract the required data from the database.
"""
def dates_to_be_included_to_answer_the_query(qry, cur_date, cach):
    sys = "You are an helpful assitant that will deduce the dates that this query is refering to."
    usr = """
    If the last purchase date is """ + cur_date
    if len(cach) == 0 :
        usr = usr + """
         
        Which dates can be included in the following query. 
        Answer me in the following format: 
        {
        "start_date": "?",
        "end_date": "?"
        } 
        If no date is specified respond with end_date equal to last purchase
        date and start_date equals 6 month before.
        If the dates can be determine do not include any explanation just the data in the specified JSON format.
        """
    else:
        usr = usr + cach +  """
         
        Which dates can be included in the following query. 
        Answer me in the following format: 
        {
        "start_date": "?",
        "end_date": "?"
        } 
        If no date is specified respond with end_date equal to last purchase
        date and start_date equals 6 month before.
        If the dates can be determine do not include any explanation just the data in the specified JSON format.
        """
    assist = "query: " + qry
    res = get_responde_from_gpt4(sys, usr, assist)
    return res


# Function that actually execute the query.

def get_answer_from_query(qry, json_context):
    sys = "You are an helpful assitant that can answer queries about my purchages including sugestions."
    usr = "To answer the question use the following data: " + json_context + """
     """
    assist = "query: " + qry
    res = get_responde_from_gpt4(sys, usr, assist, tmp= 0.5)
    return res

openai.api_key = ""
myStack = CacheStack(4, 1024)

emb_dat = pd.read_csv("data/sales_data_emb.csv")
current_date = emb_dat.iloc[-1]['dates']
while True:
# Prompt the user for a question
    question = input("What is your question? ")
    print(question)

    #If the user enters 'exit', break out of the loop
    if question == 'exit':
        break
    
    # Process the question and generate an answer
    # Is this a valid question?
    res = is_a_valid_qry(question)
    yes = "Yes".upper()
    if 3 > len(res) or res[:3].upper() != yes: # if is not yes then the question is not compatible with the app
        print(res)
        time.sleep(3)
        continue
    
    cach = myStack.convert_list_to_json()
    answer = dates_to_be_included_to_answer_the_query(question, current_date, cach)
    try:
        data = json.loads(answer)
        start_date = data['start_date']
        end_date = data['end_date']
        # Cache answer
        myStack.push(answer)
    except:
        input_date = datetime.strptime(current_date, "%m/%d/%Y")
        # Calculate the date that is 6 months before the input date
        six_months_before = input_date - timedelta(days=6*30)
        # Convert the result back to a string
        end_date = current_date
        start_date = six_months_before.strftime("%m/%d/%Y") 

    current_date_products = emb_dat.iloc[-1]['products']

    start_datetime = datetime.strptime(start_date, '%m/%d/%Y').date()
    end_datetime = datetime.strptime(end_date, '%m/%d/%Y').date()
    all_products_json = {}
    for index, row in emb_dat.iterrows():
        row_date = datetime.strptime(row['dates'], '%m/%d/%Y').date()
        if start_datetime <= row_date <= end_datetime:
            dict_prod = json.loads(row['products'])
            all_products_json.update(dict_prod)
            
# Display the answer
    json_products_concatenated = json.dumps(all_products_json)
    json_products_concatenated = cach + " " + json_products_concatenated
    answer = get_answer_from_query(question, json_products_concatenated)
    print(answer)
    prompt_response = "question:" + question + " answer: " + answer
    myStack.push(prompt_response)
    time.sleep(3) 



Tell me all the products that I bought more than once in the last semester.
To find the products bought more than once in the last semester, we will analyze the data from 12/1/2015 to 5/27/2016.

Here's the list of products bought more than once in the last semester:

1. Developer joke mug - (hip, hip, array) (White)
2. Developer joke mug - fun was unexpected at this time (Black)
3. Superhero action jacket (Blue) XXL
4. Dinosaur battery-powered slippers (Green) M
5. Dinosaur battery-powered slippers (Green) XL
6. Ogre battery-powered slippers (Green) XL
7. Permanent marker red 5mm nib (Red) 5mm
8. Superhero action jacket (Blue) XXS
9. The Gu red shirt XML tag t-shirt (White) 7XL
10. 10 mm Anti-static bubble wrap (Blue) 20m
11. Developer joke mug - that's a hardware problem (White)
12. Office cube periscope (Black)
13. Packing knife with metal insert blade (Yellow) 9mm
14. Shipping carton (Brown) 500x310x310mm
15. Alien officer hoodie (Black) 3XL
16. Alien officer hoodie (Black) XXL
17.